In [1]:
import json
import os

BASE_DIR = "/home/suyash/final_repo/HybridQA_Tab_MM/Hybrid_QA_MM"
LINK_TO_ENTITY_PATH = "/home/suyash/final_repo/HybridQA_Tab_MM/Hybrid_QA_MM/outputs_table_scraping/link_to_entity.json"


with open("/home/suyash/final_repo/Common_category_filtering/ordered_final_annotations.json") as f:
    category_annotations = json.load(f)

with open('/home/suyash/final_repo/Common_category_filtering/pageview_stats.json') as f:
    all_page_views = json.load(f)

with open('/home/suyash/final_repo/Common_category_filtering/entity_category.json') as f:
    entity_category = json.load(f)

# with open('/home/suyash/final_repo/WikiSQL/outputs/link_to_entity.json') as f:
#     link_to_entity = json.load(f)
# with open('/home/suyash/final_repo/WikiSQL/td_output/link_to_entity.json') as f:
#     temp = json.load(f)
#     link_to_entity.update(temp)

with open(LINK_TO_ENTITY_PATH) as f:
    link_to_entity = json.load(f)

with open(os.path.join(BASE_DIR, "final_dataset_unfiltered", "image_id_to_wikilink.json")) as f:
    imageid_to_wikilink = json.load(f)

wikilink_to_imageid = {}
for k,v in imageid_to_wikilink.items():
    wikilink_to_imageid[v] = k


In [2]:
from collections import defaultdict

entity_to_links = defaultdict(list)
for link, entity in link_to_entity.items():
    entity_to_links[entity].append(link)

In [3]:
len(category_annotations), len(all_page_views), len(entity_category), len(link_to_entity), len(entity_to_links)

(1423, 206448, 192826, 144354, 138668)

In [4]:
print(list(entity_category.items())[0])

('Q3235', [['Q14192234', 'mainland territory of Australia']])


In [5]:
IS_DISPLAYED = False
REMAINING_QIDS = []

def get_entities_with_category(remaining_entities, chosen_category, entity_category):
    global IS_DISPLAYED
    missing_count = 0
    relevant_entities = []
    for entity in remaining_entities:
        if entity not in entity_category:
            if not IS_DISPLAYED:
                print(entity, entity_to_links[entity])
            missing_count += 1
            REMAINING_QIDS.append(entity)
            continue
        for category in entity_category[entity]:
            if category[1] == chosen_category:
                relevant_entities.append(entity)
                break
    if not IS_DISPLAYED:
        print(f"Missing {missing_count} entities")
        IS_DISPLAYED = True
    return relevant_entities

def parse_category_type_remarks(remarks):
    seal_keywords = ['logo', 'flag', 'seal', 'coat', 'coa', "coat of arms"]
    xxx_keywords = ['xxx']
    collage_keywords = ['collage', 'location']

    if any([keyword in remarks.lower() for keyword in seal_keywords]):
        return 'seal'
    elif any([keyword in remarks.lower() for keyword in xxx_keywords]):
        return 'xxx'
    elif any([keyword in remarks.lower() for keyword in collage_keywords]):
        return 'collage'
    return 'normal'

def convert_link_to_pageview_to_qid_to_pageviews(all_page_views):
    qid_to_pageviews = defaultdict(float)
    for link, page_views in all_page_views.items():
        if link not in link_to_entity:
            continue
        qid = link_to_entity[link]
        qid_to_pageviews[qid] = max(qid_to_pageviews[qid], page_views)
    return qid_to_pageviews

In [6]:
# Iterate over the categories and parse accordingly
Is_DISPLAYED = False
REMAINING_QIDS = []

qid_to_pageviews = convert_link_to_pageview_to_qid_to_pageviews(all_page_views)

all_entities = set(list(entity_to_links.keys()))
print(len(all_entities))

seal_entities = set()
location_entities = set()
included_entities = set()
excluded_entities = set()

for category_data in category_annotations:
    category_name = category_data['category']
    is_include = category_data['include']
    limit = category_data['limit']
    remarks = category_data.get("remarks", "")
    
    relevant_entites = get_entities_with_category(all_entities, category_name, entity_category)
    annotation_type = parse_category_type_remarks(remarks)
    
    if not is_include:
        excluded_entities.update(relevant_entites)
        all_entities -= set(relevant_entites)
        continue
    
    if annotation_type == 'seal':
        for entity in relevant_entites:
            entity_page_view = qid_to_pageviews.get(entity, 0)
            if entity_page_view > limit:
                seal_entities.add(entity)
            else:
                excluded_entities.add(entity)
        
    
    if annotation_type == 'collage':
        for entity in relevant_entites:
            entity_page_view = qid_to_pageviews.get(entity, 0)
            if entity_page_view > limit:
                location_entities.add(entity)
            else:
                excluded_entities.add(entity)
    
    if annotation_type == 'xxx':
        for entity in relevant_entites:
            entity_page_view = qid_to_pageviews.get(entity, 0)
            if entity_page_view > limit:
                location_entities.add(entity)
            else:
                seal_entities.add(entity)
    
    if annotation_type == 'normal':
        for entity in relevant_entites:
            entity_page_view = qid_to_pageviews.get(entity, 0)
            if entity_page_view > limit:
                included_entities.add(entity)
            else:
                excluded_entities.add(entity)

    all_entities -= set(relevant_entites)

print("Seal entities: ", len(seal_entities))
print("Location entities: ", len(location_entities))
print("Included entities: ", len(included_entities))
print("Excluded entities: ", len(excluded_entities))
print("Entities w/o images or category:", len(set(REMAINING_QIDS)))
print("Remaining entities: ", len(all_entities)-len(set(REMAINING_QIDS)))


####OLDER OUTPUTS
# Missing 27657 entities
# Seal entities:  6291
# Location entities:  1114
# Included entities:  41236
# Excluded entities:  6137
# Entities w/o images or category: 27657
# Remaining entities:  2147


###WikiSQL Older Outputs
# Missing 75157 entities
# Seal entities:  14944
# Location entities:  1297
# Included entities:  49119
# Excluded entities:  13802
# Entities w/o images or category: 75157
# Remaining entities:  2001

## WikiSQL new output
# Missing 73885 entities
# Seal entities:  15454
# Location entities:  1299
# Included entities:  49320
# Excluded entities:  13988
# Entities w/o images or category: 73885
# Remaining entities:  2374

## HybridQA
# Missing 55881 entities
# Seal entities:  10533
# Location entities:  1159
# Included entities:  51482
# Excluded entities:  15959
# Entities w/o images or category: 55881
# Remaining entities:  3654

## AFTER GETTING CATEGORIES FOR MISSING ENTITIES
# Missing 39426 entities
# Seal entities:  12678
# Location entities:  1193
# Included entities:  61913
# Excluded entities:  18591
# Entities w/o images or category: 39426
# Remaining entities:  4867


138668
Q930236 ['https://en.wikipedia.org/wiki/Energy_harvesting']
Q1306231 ['https://en.wikipedia.org/wiki/Troy_Beyer']
Q7038204 ['https://en.wikipedia.org/wiki/Ninamaninja_Kaalpaadukal']
Q3044450 ['https://en.wikipedia.org/wiki/Honda_LPGA_Thailand', 'https://en.wikipedia.org/wiki/Honda_PTT_LPGA_Thailand']
Q8001326 ['https://en.wikipedia.org/wiki/Wildlife_Trust_for_Birmingham_and_the_Black_Country']
Q3486445 ['https://en.wikipedia.org/wiki/Skydance_Media']
Q18102 ['https://en.wikipedia.org/wiki/Saint-Mandé']
Q1971473 ['https://en.wikipedia.org/wiki/Ridolfi_plot']
Q6788333 ['https://en.wikipedia.org/wiki/Matt_Behncke']
Q7824496 ['https://en.wikipedia.org/wiki/Top_Flight']
Q159603 ['https://en.wikipedia.org/wiki/Władysław_Gomułka']
Q4961320 ['https://en.wikipedia.org/wiki/Brent_Boyd']
Q20972382 ['https://en.wikipedia.org/wiki/Mathieu_Cornet']
Q7261225 ['https://en.wikipedia.org/wiki/Puri-Gandhidham_Express']
Q4590554 ['https://en.wikipedia.org/wiki/1995_Orange_Bowl']
Q2092570 ['https://

In [7]:
############# TEMPORARY CODE #############
# Performed sanity check..... All entities that weren't there in the pageviews either didn't have an image in the infobox or didn't have category annotation in Wikidata
remaining_wikilinks = set()
count_missing_entities = 0
temp_wikilinks_set = set(wikilink_to_imageid.keys())
for qid in REMAINING_QIDS:
    for wikilink in entity_to_links[qid]:
        if wikilink in temp_wikilinks_set:
            remaining_wikilinks.add(wikilink)
print(len(remaining_wikilinks))

745


In [10]:
from pprint import pprint
# pprint(remaining_wikilinks)

remaining_qids = [link_to_entity[wikilink] for wikilink in remaining_wikilinks]
for qid in remaining_wikilinks:
    category = entity_category.get(qid, None)
    print(qid, " -> ", category)
with open('/home/suyash/final_repo/temp_scraping/qids_without_category.json', 'w') as f:
    json.dump(list(remaining_qids), f, indent=4)
with open('/home/suyash/final_repo/temp_scraping/entities_without_category_15Apr.json', 'w') as f:
    json.dump(list(remaining_wikilinks), f, indent=4)

https://en.wikipedia.org/wiki/White_Bluff_Scout_Reservation  ->  None
https://en.wikipedia.org/wiki/Fenway_Bowl  ->  None
https://en.wikipedia.org/wiki/Winter_solstice  ->  None
https://en.wikipedia.org/wiki/Redbox_Bowl  ->  None
https://en.wikipedia.org/wiki/Zarangollo  ->  None
https://en.wikipedia.org/wiki/Italian_North_Africa  ->  None
https://en.wikipedia.org/wiki/Jacksonville_Cannons  ->  None
https://en.wikipedia.org/wiki/Hamantashen  ->  None
https://en.wikipedia.org/wiki/Bayou_Bucket_Classic  ->  None
https://en.wikipedia.org/wiki/Lacón_Gallego  ->  None
https://en.wikipedia.org/wiki/SkyScreamer  ->  None
https://en.wikipedia.org/wiki/Lacoste_Ladies_Open_de_France  ->  None
https://en.wikipedia.org/wiki/Coventry_Bears  ->  None
https://en.wikipedia.org/wiki/Semifreddo  ->  None
https://en.wikipedia.org/wiki/Château_de_Chamirey  ->  None
https://en.wikipedia.org/wiki/Pitbulls  ->  None
https://en.wikipedia.org/wiki/Northern_Ireland_Trophy  ->  None
https://en.wikipedia.org/wiki

In [11]:
import os
import json

with open(os.path.join(BASE_DIR, "category_filtered_outputs", "seal_entities.json"), "w") as f:
    json.dump(list(seal_entities), f)
with open(os.path.join(BASE_DIR, "category_filtered_outputs", "location_entities.json"), "w") as f:
    json.dump(list(location_entities), f)
with open(os.path.join(BASE_DIR, "category_filtered_outputs", "included_entities.json"), "w") as f:
    json.dump(list(included_entities), f)
with open(os.path.join(BASE_DIR, "category_filtered_outputs", "excluded_entities.json"), "w") as f:
    json.dump(list(excluded_entities), f)
with open(os.path.join(BASE_DIR, "category_filtered_outputs", "remaining_entities.json"), "w") as f:
    json.dump(list(all_entities), f)


In [12]:
seal_links = []
for entity in seal_entities:
    seal_links.extend(entity_to_links[entity])

location_links = []
for entity in location_entities:
    location_links.extend(entity_to_links[entity])

included_links = []
for entity in included_entities:
    included_links.extend(entity_to_links[entity])

excluded_links = []
for entity in excluded_entities:
    excluded_links.extend(entity_to_links[entity])

remaining_links = []
for entity in all_entities:
    remaining_links.extend(entity_to_links[entity])

with open(os.path.join(BASE_DIR, "category_filtered_outputs", "seal_links.json"), "w") as f:
    json.dump(seal_links, f)
with open(os.path.join(BASE_DIR, "category_filtered_outputs", "location_links.json"), "w") as f:
    json.dump(location_links, f)
with open(os.path.join(BASE_DIR, "category_filtered_outputs", "included_links.json"), "w") as f:
    json.dump(included_links, f)
with open(os.path.join(BASE_DIR, "category_filtered_outputs", "excluded_links.json"), "w") as f:
    json.dump(excluded_links, f)
with open(os.path.join(BASE_DIR, "category_filtered_outputs", "remaining_links.json"), "w") as f:
    json.dump(remaining_links, f)
